In [1]:
import requests
import PIL
from transformers import BlipProcessor, BlipForQuestionAnswering

import pandas as pd
import numpy as np

import torch
import random

from torch.utils.data import DataLoader
from tqdm import tqdm
import pickle

from datasets import load_dataset
from datasets import Dataset
from peft import LoraConfig, get_peft_model

from sklearn.metrics import confusion_matrix

import re

2024-04-27 16:27:11.301526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/other/apps/cudnn/cudnn-linux-x86_64-8.9.7.29_cuda12-archive/lib:/opt/sw/other/apps/cuda/12.3.1/lib64:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib64:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib
2024-04-27 16:27:11.301563: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on 

In [2]:
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

torch.cuda.empty_cache()
torch.manual_seed(42)

In [4]:
device

device(type='cuda')

In [5]:
root_path = '/home/smanduru/CS682Project/data/CUB_200_2011'

images_path = root_path + '/images'
attributes_path = root_path + '/attributes/'

images = []

with open(root_path + "/images.txt", "r") as file:
    
    for line in file:
        folder, image = line.strip().split('/')
        index, folder = folder.split(" ")
        images.append(images_path + '/' + folder + '/' + image)

        
data = pd.read_csv(attributes_path + "image_attribute_labels.txt", 
                   names=['img_index', 'attribute_index', 'attribute_value', 'certainity', 'unknown'],
                   delimiter=' ', on_bad_lines='warn')

ranges = [(1, 9), (74, 79), (213, 217)] ## bill, tail, wing

df = data[data['attribute_value'] == 1]
df = df[df['attribute_index'].isin(sum([list(range(start, end + 1)) for start, end in ranges], []))]

df = df[['img_index', 'attribute_index', 'attribute_value']].reset_index(drop=True)

/tmp/ipykernel_1296105/2804928730.py:16: ParserWarning: Skipping line 709498: expected 5 fields, saw 7
Skipping line 709499: expected 5 fields, saw 7
Skipping line 709500: expected 5 fields, saw 7
Skipping line 709501: expected 5 fields, saw 7
Skipping line 709502: expected 5 fields, saw 7
Skipping line 709503: expected 5 fields, saw 7
Skipping line 709504: expected 5 fields, saw 7
Skipping line 709505: expected 5 fields, saw 7
Skipping line 709506: expected 5 fields, saw 7
Skipping line 709507: expected 5 fields, saw 7
Skipping line 709508: expected 5 fields, saw 7
Skipping line 709509: expected 5 fields, saw 7
Skipping line 709510: expected 5 fields, saw 7
Skipping line 709511: expected 5 fields, saw 7
Skipping line 709512: expected 5 fields, saw 7
Skipping line 709513: expected 5 fields, saw 7
Skipping line 709514: expected 5 fields, saw 7
Skipping line 709515: expected 5 fields, saw 7
Skipping line 709516: expected 5 fields, saw 7
Skipping line 709517: expected 5 fields, saw 7
Skip

/tmp/ipykernel_1296105/2804928730.py:16: ParserWarning: Skipping line 2921266: expected 5 fields, saw 7
Skipping line 2921267: expected 5 fields, saw 7
Skipping line 2921268: expected 5 fields, saw 7
Skipping line 2921269: expected 5 fields, saw 7
Skipping line 2921270: expected 5 fields, saw 7
Skipping line 2921271: expected 5 fields, saw 7
Skipping line 2921272: expected 5 fields, saw 7
Skipping line 2921273: expected 5 fields, saw 7
Skipping line 2921274: expected 5 fields, saw 7
Skipping line 2921275: expected 5 fields, saw 7
Skipping line 2921276: expected 5 fields, saw 7
Skipping line 2921277: expected 5 fields, saw 7
Skipping line 2921278: expected 5 fields, saw 7
Skipping line 2921279: expected 5 fields, saw 7
Skipping line 2921280: expected 5 fields, saw 7
Skipping line 2921281: expected 5 fields, saw 7
Skipping line 2921282: expected 5 fields, saw 7
Skipping line 2921283: expected 5 fields, saw 7
Skipping line 2921284: expected 5 fields, saw 7
Skipping line 2921285: expected 

In [26]:
attributes = {}
with open('/home/smanduru/CS682Project/data/attributes.txt' , "r") as file:
    
    for line in file:
        
        attribute_index, attribute = line.split()
        attribute_qsn, attribute_value = attribute.split('::')
        attribute_qsn = attribute_qsn.split('_', 1)[-1]
        attribute_qsn = attribute_qsn.replace('_', ' ')
        
        attribute_value = re.sub(r'[^a-zA-Z0-9\s]', ' ', attribute_value)
        attribute_value = re.sub(r'\s+', ' ', attribute_value)
        attributes[attribute_index] = [int(attribute_index), attribute_qsn, attribute_value]

att_df = pd.DataFrame.from_dict(attributes, orient='index', 
                                columns=['attribute_index', 'attribute_qsn', 'attribute_answer'])


merged_df = df.merge(att_df, how='inner', on='attribute_index')
merged_df = merged_df.sort_values(by=['img_index', 'attribute_index']).reset_index(drop = True)

In [27]:
merged_df

,img_index,attribute_index,attribute_value,attribute_qsn,attribute_answer
0,1,5,1,bill shape,hooked seabird
1,2,76,1,tail shape,notched tail
2,2,213,1,wing shape,rounded wings
3,3,5,1,bill shape,hooked seabird
4,3,217,1,wing shape,long wings
...,...,...,...,...,...
28760,11787,76,1,tail shape,notched tail
28761,11787,213,1,wing shape,rounded wings
28762,11788,7,1,bill shape,all purpose
28763,11788,78,1,tail shape,pointed tail


In [28]:
merged_df['attribute_answer'].value_counts()

attribute_answer
all purpose           4618
notched tail          3505
rounded wings         3291
cone                  3082
pointed wings         2352
pointed tail          2090
dagger                1591
rounded tail          1301
squared tail          1027
tapered wings          960
fan shaped tail        914
hooked seabird         751
long wings             711
broad wings            699
spatulate              469
forked tail            417
needle                 289
curved up or down      259
specialized            251
hooked                 188
Name: count, dtype: int64

In [29]:
bboxes = []
with open('/home/smanduru/CS682Project/data/CUB_200_2011/bounding_boxes.txt' , "r") as file:
    for line in file:
        if line:
            _, bbox = line.strip().split(" ", 1)
            x, y, w, h = map(int, map(float, bbox.split()))
            bbox = [x, y, w, h]
            bboxes.append(bbox)

len(bboxes)

11788

In [30]:
answer_candidates = ["blue", "brown", "iridescent", "purple",
                     "rufous", "grey", "yellow", "olive",
                     "green", "pink", "orange", "black",
                     "white", "red", "buff"]

In [31]:
target_shape = (224, 224, 3)
class VQADataset(torch.utils.data.Dataset):
    """Caltech Bird Dataset"""

    def __init__(self, dataset, processor, images):
        self.dataset = dataset
        self.processor = processor
        self.images = images

    def __len__(self):
        return len(self.dataset)
    
    def preprocess_image(image_path, bounding_box, target_shape):
        x, y, w, h = map(int, bounding_box)
        
        cropped_image = image[y:y+h, x:x+w]
        resized_image = cv2.resize(cropped_image, target_shape[:2])  # Resize to target shape
        normalized_image = resized_image / 255.0  # Normalize pixel values
        return normalized_image

    def __getitem__(self, idx):
        # get image + text
        question_part = self.dataset[idx]['attribute_qsn']
        question = f"What is the {question_part} of the bird?"
        answer = self.dataset[idx]['attribute_answer']
        
        img_index = self.dataset[idx]['img_index']
        image_path = images[int(img_index) - 1]
        x, y, w, h = bboxes[int(img_index) - 1]
        image = PIL.Image.open(image_path).convert("RGB")
        cropped_image = image.crop((x, y, x + w, y + h))  # Crop image based on bounding box
        resized_image = cropped_image.resize(target_shape[:2], PIL.Image.Resampling.LANCZOS)
        image = np.array(resized_image) / 255.0
        
        text = question
        encoding = self.processor(image, text, padding="max_length", truncation=True, return_tensors="pt")
        #labels = self.processor.tokenizer.encode(
        #    answer, max_length= 8, pad_to_max_length=True, return_tensors='pt'
        #)
        labels = self.processor(text = answer, max_length= 8, pad_to_max_length=True, 
                                return_tensors="pt").input_ids
        encoding["labels"] = labels
        # remove batch dimension
        for k,v in encoding.items():  encoding[k] = v.squeeze()
        return encoding

In [32]:
dataset = Dataset.from_pandas(merged_df).train_test_split(test_size=0.2)

In [33]:
dataset

DatasetDict({
    train: Dataset({
        features: ['img_index', 'attribute_index', 'attribute_value', 'attribute_qsn', 'attribute_answer'],
        num_rows: 23012
    })
    test: Dataset({
        features: ['img_index', 'attribute_index', 'attribute_value', 'attribute_qsn', 'attribute_answer'],
        num_rows: 5753
    })
})

In [34]:
# training_dataset = load_dataset("json", data_files="Data/train.jsonl", split="train[:90%]")
# valid_dataset = load_dataset("json", data_files="Data/train.jsonl", split="train[90%:]")
# print("Training sets: {} - Validating set: {}".format(len(training_dataset), len(valid_dataset)))


train_dataset = VQADataset(dataset=dataset['train'],
                          processor=processor, images = images)
valid_dataset = VQADataset(dataset=dataset['test'],
                          processor=processor, images = images)


In [35]:
batch_size = 12
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Zero Shot 

In [36]:
# blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
# blip_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")


# total_correct = total_samples = 0
# for batch in tqdm(valid_dataloader, desc='Validating'):
#     inputs = batch.to(device)
#     labels = batch['labels'].to(device)
#     outputs = model.generate(**inputs)

#     actuals = [processor.tokenizer.decode(label, skip_special_tokens=True) for label in labels]
#     predicted = [processor.decode(out, skip_special_tokens=True) for out in outputs]
#     total_correct += sum(1 for pred, actual in zip(predicted, actuals) if pred == actual)
#     total_samples += labels.size(0)

# accuracy = total_correct / total_samples
# print('Accuracy on validation set:', accuracy)
# Accuracy on validation set: 0.20355053459753883

# Finetuning the Blip

In [37]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 387,031,868 || trainable%: 0.6095870120958619


In [39]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1, verbose=False)
num_epochs = 1
patience = 10
min_eval_loss = float("inf")
early_stopping_hook = 0
tracking_information = []
scaler = torch.cuda.amp.GradScaler()

In [40]:
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()
    for idx, batch in zip(tqdm(range(len(train_dataloader)), desc='Training batch: ...'), train_dataloader):
        input_ids = batch.pop('input_ids').to(device)
        pixel_values = batch.pop('pixel_values').to(device)
        attention_masked = batch.pop('attention_mask').to(device)
        labels = batch.pop('labels').to(device)
        
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        attention_mask=attention_masked,
                        labels=labels)
            
        loss = outputs.loss
        epoch_loss += loss.item()
        # loss.backward()
        # optimizer.step()
        optimizer.zero_grad()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    
    model.eval()
    eval_loss = 0
    for idx, batch in zip(tqdm(range(len(valid_dataloader)), desc='Validating batch: ...'), valid_dataloader):
        input_ids = batch.pop('input_ids').to(device)
        pixel_values = batch.pop('pixel_values').to(device)
        attention_masked = batch.pop('attention_mask').to(device)
        labels = batch.pop('labels').to(device)

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        attention_mask=attention_masked,
                        labels=labels)

        loss = outputs.loss
        eval_loss += loss.item()
    
    tracking_information.append((epoch_loss/len(train_dataloader), 
                                 eval_loss/len(valid_dataloader), 
                                 optimizer.param_groups[0]["lr"]))
    print("Epoch: {} - Training loss: {} - Eval Loss: {} - LR: {}".format(epoch+1,
                                                                          epoch_loss/len(train_dataloader),
                                                                          eval_loss/len(valid_dataloader),
                                                                          optimizer.param_groups[0]["lr"]))
    scheduler.step()
    if eval_loss < min_eval_loss:
        model.save_pretrained("Model/temp_blip-saved-model", from_pt=True) 
        print("Saved model to Model/blip-saved-model")
        min_eval_loss = eval_loss
        early_stopping_hook = 0
    else:
        early_stopping_hook += 1
        if early_stopping_hook > patience:
            break

Training batch: ...:   0%|          | 0/1918 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/smanduru/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Validating batch: ...: 100%|

Epoch: 1 - Training loss: 2.06950957092677 - Eval Loss: 2.000851273039977 - LR: 0.0005
Saved model to Model/blip-saved-model


In [41]:
pickle.dump(tracking_information, open("temp_tracking_information.pkl", "wb"))
print("The finetuning process has done!")

The finetuning process has done!


In [42]:
# # from transformers import AutoModel

# # Load the saved model
# model = AutoModel.from_pretrained("Model/blip-saved-model")

# Evaluation on Valid Dataset

In [43]:
import nltk
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import numpy as np
import evaluate


def wup(word1, word2, alpha):
    """
    calculate the wup similarity
    :param word1:
    :param word2:
    :param alpha:
    :return:
    """
    # print(word1, word2)
    if word1 == word2:
        return 1.0

    w1 = wordnet.synsets(word1)
    w1_len = len(w1)
    if w1_len == 0: return 0.0
    w2 = wordnet.synsets(word2)
    w2_len = len(w2)
    if w2_len == 0: return 0.0

    #match the first
    word_sim = w1[0].wup_similarity(w2[0])
    if word_sim is None:
        word_sim = 0.0

    if word_sim < alpha:
        word_sim = 0.1*word_sim
    return word_sim


def wups(words1, words2, alpha):
    """

    :param pred:
    :param truth:
    :param alpha:
    :return:
    """
    sim = 1.0
    flag = False
    for w1 in words1:
        max_sim = 0
        for w2 in words2:
            word_sim = wup(w1, w2, alpha)
            if word_sim > max_sim:
                max_sim = word_sim
        if max_sim == 0: continue
        sim *= max_sim
        flag = True
    if not flag:
        sim = 0.0
    return sim


def get_wups(pred, truth, alpha):
    """
    calculate the wups score
    :param pred:
    :param truth:
    :return:
    """
    pred = word_tokenize(pred)
    truth = word_tokenize(truth)
    item1 = wups(pred, truth, alpha)
    item2 = wups(truth, pred, alpha)
    value = min(item1, item2)
    return value

def levenshtein_distance(s1, s2):
    """Calculates the Levenshtein distance between two strings.


    The Levenshtein distance is defined as the number of additions, deletions or substitutions required to transform one string into another. This can be thought of as a measure of similarity between two sequences. It differs from the standard edit distance in that it allows for transpositions and replaces all insertions with deletions.


    Args:
        s1 (str): first string
        s2 (str): second string


    Returns:
        int: Levenshtein distance between `s1` and `s2`. If both are equal then 0 is returned.
    """
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)


    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)


    previous_row = range(len(s2)+1)
    for i, c1 in enumerate(s1):
        current_row = [i+1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1!= c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

[nltk_data] Downloading package omw-1.4 to /home/smanduru/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [44]:
model.eval()  # Set the model to evaluation mode

total_correct = 0
total_samples = 0

all_actuals = []
all_predicted = []
all_wups = []
all_lds = []

with torch.no_grad():
    for batch in tqdm(valid_dataloader, desc='Validating'):
        inputs = batch.to(device)
        labels = batch['labels'].to(device)
        outputs = model.generate(**inputs)

        predicted = processor.batch_decode(outputs, skip_special_tokens=True)
        actuals = processor.batch_decode(labels, skip_special_tokens=True)
        _wups = [get_wups(ans, act, 0.9) for ans, act in zip(predicted, actuals)]
        _ld = [levenshtein_distance(ans, act) for ans, act in zip(predicted, actuals)]
        
        total_correct += sum(1 for pred, actual in zip(predicted, actuals) if pred == actual)
        total_samples += labels.size(0)
        
        all_actuals.extend(actuals)
        all_predicted.extend(predicted)
        all_wups.extend(_wups)
        all_lds.extend(_ld)

# Compute confusion matrix
conf_matrix = confusion_matrix(all_actuals, all_predicted)

# Print confusion matrix
# print("Confusion Matrix:")
# print(conf_matrix)
print("WUPS and Levenshtein Distance: ", sum(all_wups)/len(all_wups), sum(all_lds)/len(all_lds))

# Calculate overall accuracy
accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
print('Accuracy on validation set:', accuracy, total_correct/total_samples)

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=all_predicted, references=all_actuals)
print('BLEU', results['bleu'])

# Accuracy on validation set: 0.369780108936857
# Accuracy on validation set: 0.37088965099858784
# Accuracy on validation set: 0.43473875327819245 on finetuning bboxes


Validating:   0%|          | 0/480 [00:00<?, ?it/s]/home/smanduru/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Validating: 100%|██████████| 480/480 [06:55<00:00,  1.15it/s]


WUPS and Levenshtein Distance:  0.2833167910882376 6.043281766035112
Accuracy on validation set: 0.1586998087954111 0.1586998087954111
BLEU 0.0
